# Import Git

In [1]:
!git clone https://github.com/KhoiDang-KHDL/DS201_Lab03-RNN


Cloning into 'DS201_Lab03-RNN'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 25 (delta 10), reused 17 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 1.09 MiB | 7.81 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [2]:
import sys
sys.path.append('/kaggle/working/DS201_Lab03-RNN') 

# Check thư mục
import os
print(os.listdir('/kaggle/working/DS201_Lab03-RNN'))

['main.ipynb', 'PhoNer.py', 'lstm.py', 'uit_vsfc.py', '.git', 'seq2seq.py', 'UIT-VSFC-20251121T013658Z-1-001', 'PhoNER', 'gru.py']


# Thư viện

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import uit_vsfc

from uit_vsfc import Vocab, UIT_VSFC, collate_fn
from lstm import LSTMModel
from gru import GRUModel

# Cấu hình cho bài 1,2

In [4]:
BATCH_SIZE = 64
HIDDEN_SIZE = 128  
N_LAYERS = 2
LEARNING_RATE = 0.001
EPOCHS = 10
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train, Eval bài 1,2

In [5]:
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(DEVICE)
            labels = batch['label'].to(DEVICE)
            
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            
            # Lấy nhãn dự đoán
            _, predicted = torch.max(outputs, 1)
            
            # Chuyển về cpu để tính trong sklearn
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    avg_loss = total_loss / len(dataloader)
    
    # Tính Accuracy và F1-Score
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return avg_loss, acc, f1

In [6]:
def train(model, train_loader, dev_loader, optimizer, criterion, epochs):
    print(f"Bắt đầu huấn luyện trên thiết bị: {DEVICE}")
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for i, batch in enumerate(train_loader):
            input_ids = batch['input_ids'].to(DEVICE)
            labels = batch['label'].to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            if (i + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        dev_loss, dev_acc, dev_f1 = evaluate(model, dev_loader, criterion)
        print(f"--- Hết Epoch {epoch+1} ---")
        print(f"Train Loss: {running_loss/len(train_loader):.4f}")
        print(f"Dev Loss: {dev_loss:.4f} | Dev Acc: {dev_acc*100:.2f}% | Dev F1: {dev_f1*100:.2f}%")
        print("-" * 30)

# Bài 1: main

In [7]:
def main():

    base_path = r'/kaggle/working/DS201_Lab03-RNN/UIT-VSFC-20251121T013658Z-1-001/UIT-VSFC'
    
    train_path = os.path.join(base_path, 'UIT-VSFC-train.json')
    dev_path = os.path.join(base_path, 'UIT-VSFC-dev.json')
    test_path = os.path.join(base_path, 'UIT-VSFC-test.json')

    print("Đang xây dựng bộ từ điển (Vocab):")
    vocab = Vocab(base_path)
    print(f"Kích thước từ điển: {vocab.len}")

    print("Đang tải dữ liệu...")
    train_dataset = UIT_VSFC(train_path, vocab)
    dev_dataset = UIT_VSFC(dev_path, vocab)
    test_dataset = UIT_VSFC(test_path, vocab)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    # --- MODEL ---
    model = LSTMModel(
        vocab_size=vocab.len,
        hidden_size=HIDDEN_SIZE,
        n_layers=N_LAYERS,
        n_labels=vocab.n_labels,
        padding_idx=vocab.w2i[vocab.pad]
    ).to(DEVICE)

    # ---  OPTIMIZER, LOSS ---
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train(model, train_loader, dev_loader, optimizer, criterion, EPOCHS)

    # Evaluate
    print("\nĐang đánh giá trên tập Test...")
    test_loss, test_acc, test_f1 = evaluate(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc*100:.2f}%")
    print(f"Test F1-Score: {test_f1*100:.2f}%")

if __name__ == '__main__':
    main()

Đang xây dựng bộ từ điển (Vocab)...
Kích thước từ điển: 2848
Đang tải dữ liệu...
Bắt đầu huấn luyện trên thiết bị: cuda
Epoch [1/10], Step [10/179], Loss: 1.1633
Epoch [1/10], Step [20/179], Loss: 0.7219
Epoch [1/10], Step [30/179], Loss: 0.9660
Epoch [1/10], Step [40/179], Loss: 0.9687
Epoch [1/10], Step [50/179], Loss: 0.8883
Epoch [1/10], Step [60/179], Loss: 0.7369
Epoch [1/10], Step [70/179], Loss: 0.7962
Epoch [1/10], Step [80/179], Loss: 0.7773
Epoch [1/10], Step [90/179], Loss: 0.8076
Epoch [1/10], Step [100/179], Loss: 0.8188
Epoch [1/10], Step [110/179], Loss: 0.8830
Epoch [1/10], Step [120/179], Loss: 1.1580
Epoch [1/10], Step [130/179], Loss: 0.8586
Epoch [1/10], Step [140/179], Loss: 0.8490
Epoch [1/10], Step [150/179], Loss: 0.8769
Epoch [1/10], Step [160/179], Loss: 0.9175
Epoch [1/10], Step [170/179], Loss: 0.9098
--- Hết Epoch 1 ---
Train Loss: 0.8706
Dev Loss: 0.8377 | Dev Acc: 72.65% | Dev F1: 61.21%
------------------------------
Epoch [2/10], Step [10/179], Loss: 0

# Bài 2: main

In [8]:
def main():

    base_path = r'/kaggle/working/DS201_Lab03-RNN/UIT-VSFC-20251121T013658Z-1-001/UIT-VSFC'
    
    train_path = os.path.join(base_path, 'UIT-VSFC-train.json')
    dev_path = os.path.join(base_path, 'UIT-VSFC-dev.json')
    test_path = os.path.join(base_path, 'UIT-VSFC-test.json')

    print("Đang xây dựng bộ từ điển (Vocab)...")
    vocab = Vocab(base_path)
    print(f"Kích thước từ điển: {vocab.len}")

    print("Đang tải dữ liệu...")
    train_dataset = UIT_VSFC(train_path, vocab)
    dev_dataset = UIT_VSFC(dev_path, vocab)
    test_dataset = UIT_VSFC(test_path, vocab)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    # ---  MODEL ---
    model = GRUModel(
        vocab_size=vocab.len,
        embedding_dim=128,
        hidden_size=HIDDEN_SIZE,
        n_layers=N_LAYERS,
        n_labels=vocab.n_labels,
        padding_idx=vocab.w2i[vocab.pad]
    ).to(DEVICE)

    # ---  OPTIMIZER, LOSS ---
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train(model, train_loader, dev_loader, optimizer, criterion, EPOCHS)
    
    # Evaluate
    print("\nĐang đánh giá trên tập Test...")
    test_loss, test_acc, test_f1 = evaluate(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc*100:.2f}%")
    print(f"Test F1-Score: {test_f1*100:.2f}%")

if __name__ == '__main__':
    main()

Đang xây dựng bộ từ điển (Vocab)...
Kích thước từ điển: 2848
Đang tải dữ liệu...
Bắt đầu huấn luyện trên thiết bị: cuda
Epoch [1/10], Step [10/179], Loss: 0.9440
Epoch [1/10], Step [20/179], Loss: 0.7779
Epoch [1/10], Step [30/179], Loss: 0.5210
Epoch [1/10], Step [40/179], Loss: 0.4982
Epoch [1/10], Step [50/179], Loss: 0.4534
Epoch [1/10], Step [60/179], Loss: 0.7262
Epoch [1/10], Step [70/179], Loss: 0.3585
Epoch [1/10], Step [80/179], Loss: 0.4916
Epoch [1/10], Step [90/179], Loss: 0.4982
Epoch [1/10], Step [100/179], Loss: 0.4240
Epoch [1/10], Step [110/179], Loss: 0.3566
Epoch [1/10], Step [120/179], Loss: 0.4209
Epoch [1/10], Step [130/179], Loss: 0.2956
Epoch [1/10], Step [140/179], Loss: 0.2559
Epoch [1/10], Step [150/179], Loss: 0.2795
Epoch [1/10], Step [160/179], Loss: 0.4488
Epoch [1/10], Step [170/179], Loss: 0.3230
--- Hết Epoch 1 ---
Train Loss: 0.4708
Dev Loss: 0.3684 | Dev Acc: 86.10% | Dev F1: 85.99%
------------------------------
Epoch [2/10], Step [10/179], Loss: 0

# Bài 3

## Import module

In [ ]:
from PhoNer import Vocab, PhoNER, collate_fn 
from seq2seq import Encoder, Decoder, Seq2Seq

## Cấu hình

In [ ]:
# --- CONFIG ---
BATCH_SIZE = 32 
ENC_EMB_DIM = 100
DEC_EMB_DIM = 100
HIDDEN_DIM = 256 
N_LAYERS = 5       
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
EPOCHS = 10
LEARNING_RATE = 0.001
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Train, Eval bài 3

In [ ]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    model.eval()
    epoch_loss = 0
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in iterator:
            src = batch['input_ids'].to(DEVICE)
            trg = batch['label'].to(DEVICE)

            # Tắt teacher forcing khi evaluate (ratio = 0)
            output = model(src, trg, 0) 
            
            # output: [batch, seq_len, output_dim]
            # trg: [batch, seq_len]
            
            # Reshape để tính loss
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim) # Bỏ token đầu
            trg = trg[:, 1:].reshape(-1)                   # Bỏ token đầu
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            
            # --- TÍNH F1 ---
            # Lấy nhãn dự đoán
            preds = output.argmax(dim=1)
            
            # Chuyển về CPU list
            preds_list = preds.cpu().numpy()
            trg_list = trg.cpu().numpy()
            
            # Lọc bỏ các giá trị Padding trong trg để tính F1 chính xác
            # Vì trg có pad_idx là -100
            valid_indices = np.where(trg_list != tag_pad_idx)[0]
            
            all_preds.extend(preds_list[valid_indices])
            all_labels.extend(trg_list[valid_indices])
            
    avg_loss = epoch_loss / len(iterator)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return avg_loss, f1

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch['input_ids'].to(DEVICE)
        trg = batch['label'].to(DEVICE)
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        # Reshape output và trg để tính CrossEntropy
        # output shape: [batch, seq_len, n_tags] -> flatten -> [batch * seq, n_tags]
        # trg shape: [batch, seq_len] -> flatten -> [batch * seq]
        
        # Lưu ý: Bỏ qua phần tử đầu tiên (index 0) vì Seq2Seq start loop từ 1
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        # Cắt gradient để tránh bùng nổ gradient (exploding gradient) với LSTM nhiều lớp
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += loss.item()
        
        if (i+1) % 10 == 0:
             print(f"Step {i+1}/{len(iterator)} | Loss: {loss.item():.4f}")
        
    return epoch_loss / len(iterator)

## Main bài 3

In [9]:
def main():
    # 1. Load Data
    # Sửa đường dẫn tới thư mục chứa file json của PhoNER
    base_path = r'/kaggle/working/DS201_Lab03-RNN/PhoNER' 
    
    print("Đang đọc dữ liệu...")
    vocab = Vocab(base_path)
    print(f"Vocab size: {vocab.len} | Tags: {vocab.n_labels}")
    
    train_data = PhoNER(os.path.join(base_path, 'train_word.json'), vocab)
    dev_data = PhoNER(os.path.join(base_path, 'dev_word.json'), vocab)
    test_data = PhoNER(os.path.join(base_path, 'test_word.json'), vocab)
    
    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
    
    # 2. Khởi tạo Model
    input_dim = vocab.len
    output_dim = vocab.n_labels
    
    enc = Encoder(input_dim, ENC_EMB_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT)
    dec = Decoder(output_dim, DEC_EMB_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT)
    model = Seq2Seq(enc, dec, DEVICE).to(DEVICE)
    
    # Init weights (tùy chọn, giúp hội tụ tốt hơn)
    def init_weights(m):
        for name, param in m.named_parameters():
            nn.init.uniform_(param.data, -0.08, 0.08)
    model.apply(init_weights)

    print(f'Mô hình có {sum(p.numel() for p in model.parameters() if p.requires_grad):,} tham số train được.')

    # 3. Optimizer & Loss
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # ignore_index=-100 để không tính loss cho phần padding
    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    
    # 4. Training Loop
    print("Bắt đầu huấn luyện...")
    best_valid_loss = float('inf')
    
    for epoch in range(EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion, clip=1)
        valid_loss, valid_f1 = evaluate(model, dev_loader, criterion, tag_pad_idx=-100)
        
        print(f'Epoch: {epoch+1:02}')
        print(f'\tTrain Loss: {train_loss:.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. F1: {valid_f1*100:.2f}%')
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'phoner_seq2seq.pt')
    
    # 5. Test
    print("\nĐang đánh giá trên tập Test...")
    model.load_state_dict(torch.load('phoner_seq2seq.pt'))
    test_loss, test_f1 = evaluate(model, test_loader, criterion, tag_pad_idx=-100)
    print(f'Test Loss: {test_loss:.3f} | Test F1: {test_f1*100:.2f}%')

if __name__ == '__main__':
    main()

Đang đọc dữ liệu...
--- Đang quét dữ liệu tại: /kaggle/working/DS201_Lab03-RNN/PhoNER ---
-> Đã xây dựng Vocab: 7306 từ, 21 nhãn (bao gồm pad).
Vocab size: 7306 | Tags: 21
-> Đã tải 5027 mẫu từ train_word.json
-> Đã tải 2000 mẫu từ dev_word.json
-> Đã tải 3000 mẫu từ test_word.json
Mô hình có 5,681,969 tham số train được.
Bắt đầu huấn luyện...
Step 10/158 | Loss: 0.8964
Step 20/158 | Loss: 0.7721
Step 30/158 | Loss: 0.8356
Step 40/158 | Loss: 0.8225
Step 50/158 | Loss: 0.6643
Step 60/158 | Loss: 0.7339
Step 70/158 | Loss: 0.7028
Step 80/158 | Loss: 0.6225
Step 90/158 | Loss: 0.7005
Step 100/158 | Loss: 0.7773
Step 110/158 | Loss: 0.6410
Step 120/158 | Loss: 0.6180
Step 130/158 | Loss: 0.4686
Step 140/158 | Loss: 0.6367
Step 150/158 | Loss: 0.5616
Epoch: 01
	Train Loss: 0.745
	 Val. Loss: 1.451 |  Val. F1: 15.11%
Step 10/158 | Loss: 0.6464
Step 20/158 | Loss: 0.5114
Step 30/158 | Loss: 0.3016
Step 40/158 | Loss: 0.4225
Step 50/158 | Loss: 0.6511
Step 60/158 | Loss: 0.7583
Step 70/158 | 

# 